## Ford GoBike's trip data

https://www.fordgobike.com/system-data


Each trip is anonymized and includes:
- Trip Duration (seconds)
- Start Time and Date
- End Time and Date
- Start Station ID
- Start Station Name
- Start Station Latitude
- Start Station Longitude
- End Station ID
- End Station Name
- End Station Latitude
- End Station Longitude
- Bike ID
- User Type (Subscriber or Customer – “Subscriber” = Member or “Customer” = Casual)
- Member Year of Birth
- Member Gender


In [1]:
val rawData = sc.textFile("2017-fordgobike-tripdata.csv")
rawData.take(2)

Intitializing Scala interpreter ...

Spark Web UI available at http://ce7d2534405e:4040
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1597326676544)
SparkSession available as 'spark'


rawData: org.apache.spark.rdd.RDD[String] = 2017-fordgobike-tripdata.csv MapPartitionsRDD[1] at textFile at <console>:25
res0: Array[String] = Array("duration_sec","start_time","end_time","start_station_id","start_station_name","start_station_latitude","start_station_longitude","end_station_id","end_station_name","end_station_latitude","end_station_longitude","bike_id","user_type","member_birth_year","member_gender", "80110","2017-12-31 16:57:39.6540","2018-01-01 15:12:50.2450","74","Laguna St at Hayes St","37.776434819204745","-122.42624402046204","43","San Francisco Public Library (Grove St at Hyde St)","37.7787677","-122.4159292","96","Customer","1987","Male")


In [2]:
val colNames = rawData.filter(_.startsWith("\"duration")).flatMap(_.split(",")).map(x => x.slice(1, x.length -1)).collect()

colNames.foreach(println)

duration_sec
start_time
end_time
start_station_id
start_station_name
start_station_latitude
start_station_longitude
end_station_id
end_station_name
end_station_latitude
end_station_longitude
bike_id
user_type
member_birth_year
member_gender


colNames: Array[String] = Array(duration_sec, start_time, end_time, start_station_id, start_station_name, start_station_latitude, start_station_longitude, end_station_id, end_station_name, end_station_latitude, end_station_longitude, bike_id, user_type, member_birth_year, member_gender)


### > **TODO**: Utwórz RDD `transData` gdzie elementami są listy stringów (odfiltruj element zawierający nazwy)

In [3]:
val transData = rawData.filter(!_.startsWith("\"duration")).map(_.split(",")).map(_.map(x => x.slice(1, x.length -1)))
transData.take(1)(0)

transData: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[7] at map at <console>:26
res2: Array[String] = Array(80110, 2017-12-31 16:57:39.6540, 2018-01-01 15:12:50.2450, 74, Laguna St at Hayes St, 37.776434819204745, -122.42624402046204, 43, San Francisco Public Library (Grove St at Hyde St), 37.7787677, -122.4159292, 96, Customer, 1987, Male)


> **TODO**: Sprawdź czy, gdzie i w ilu rekordach występują braki danych w RDD `transData` (hint: elementami RDD są listy stringów)

In [4]:
transData.map(_.contains("")).count()

res3: Long = 519700


> **TODO**: Uwzględniając informacje uzyskane w poprzednim ćwiczeniu utwórz RDD `data` którego elementy będą słownikami. Słowniki te mają zawierać klucze o wartościach następujących typów:
- 'duration_sec' : "int"
- 'start_time' : "datetime"
- 'end_time' : "datetime"
- 'start_station_id' : "int"
- 'start_station_name' : "string"
- 'start_station_latitude' : "float"
- 'start_station_longitude' : "float"
- 'end_station_id' : "int"
- 'end_station_name' : "string"
- 'end_station_latitude' : "float"
- 'end_station_longitude' : "float"
- 'bike_id' : "int"
- 'user_type' : "string"
- 'member_birth_year' : "int"
- 'member_gender' : "string"

> (hint: autor zadania rozwiązał je pisząc funkcję przekształcającą elementy w RDD)

In [5]:
val struct = Array("Int", 
          "Datetime", 
          "Datetime", 
          "Int", 
          "String", 
          "Float", 
          "Float", 
          "Int", 
          "String", 
          "Float", 
          "Float", 
          "Int", 
          "String", 
          "Int", 
          "String")

struct.mkString("\n")

struct: Array[String] = Array(Int, Datetime, Datetime, Int, String, Float, Float, Int, String, Float, Float, Int, String, Int, String)
res4: String =
Int
Datetime
Datetime
Int
String
Float
Float
Int
String
Float
Float
Int
String
Int
String


In [6]:
  import java.time.LocalDateTime
  import java.time.format.DateTimeFormatter
  import java.time.temporal.ChronoUnit

import java.time.LocalDateTime
import java.time.format.DateTimeFormatter
import java.time.temporal.ChronoUnit


In [7]:
  def zipMany[X,Y,Z](xs: Seq[X], ys: Seq[Y], zs: Seq[Z]): Seq[(X,Y,Z)] =
    for (((x, y), z) <- xs zip ys zip zs) yield (x, y, z)

zipMany: [X, Y, Z](xs: Seq[X], ys: Seq[Y], zs: Seq[Z])Seq[(X, Y, Z)]


In [8]:
 def transform_element (rdd_elem: Array[String], elem_struct: Array[String], names: Array[String]) = {
    val DATE_FORMATTER = DateTimeFormatter.ofPattern("yyyy-MM-dd HH:mm:ss.SSSS")
    var map = scala.collection.mutable.Map[String, Any]()

    val zipped = zipMany(rdd_elem, elem_struct, names)

    for ((elem, e_type, name) <- zipped) {
      e_type match  {
        case "Int" => elem match {
          case "" => map += (name -> 9999)
          case _ => map += (name -> elem.toInt)
        }
        case "Datetime" => map += (name -> LocalDateTime.parse(elem, DATE_FORMATTER))
        case "Float" => map += (name -> elem.toFloat)
        case _ => map += (name -> elem)
      }
    }
    map
  }

transform_element: (rdd_elem: Array[String], elem_struct: Array[String], names: Array[String])scala.collection.mutable.Map[String,Any]


In [9]:
 val data = transData.map(x => transform_element(x, struct, colNames))

data: org.apache.spark.rdd.RDD[scala.collection.mutable.Map[String,Any]] = MapPartitionsRDD[9] at map at <console>:35


In [10]:
data.take(1)(0).mkString("\n")

res5: String =
start_station_id -> 74
member_gender -> Male
start_station_longitude -> -122.42625
end_station_latitude -> 37.778767
member_birth_year -> 1987
start_station_name -> Laguna St at Hayes St
end_station_longitude -> -122.41593
start_time -> 2017-12-31T16:57:39.654
end_time -> 2018-01-01T15:12:50.245
duration_sec -> 80110
user_type -> Customer
end_station_name -> San Francisco Public Library (Grove St at Hyde St)
start_station_latitude -> 37.776436
bike_id -> 96
end_station_id -> 43


In [11]:
data.cache()

res6: data.type = MapPartitionsRDD[9] at map at <console>:35


> **ZADANIE 1**: Zapoznaj się z danymi:
1. Sprawdź rozkład zmiennej "member_gender"
2. Oblicz minimalny, maksymalny i średni wiek osób wypożyczających rowery
3. Oblicz liczbę unikalnych rowerów
4. Oblicz liczbę unikalnych stacji
5. Sprawdź który rower był wypożyczony najdłużej a który najkrócej w ciągu analizowanego okresu (oraz jak długo)
6. Oblicz średni czas pojedynczego wypożyczenia
7. Sprawdź pomiędzy którymi stacjami występował największy ruch
8. Sprawdź o której godzinie w ciągu dnia wypożyczano najwięcej rowerów
9. Sprawdź *średnią liczbę wypożyczeń* dla poszczególnych dni tygodnia

In [12]:
//1
val member_gender =  data.map(x => (x("member_gender"),1)).reduceByKey(_ + _).collect()

member_gender: Array[(Any, Int)] = Array((Other,6299), ("",66462), (Female,98621), (Male,348318))


In [13]:
//2
val min_year = data.filter(x => x("member_birth_year") != 9999
                          ).map(x => 2020 - x("member_birth_year").asInstanceOf[Int]).stats()

min_year: org.apache.spark.util.StatCounter = (count: 453159, mean: 39.595213, stdev: 10.513476, max: 134.000000, min: 21.000000)


In [14]:
//3
data.map(_("bike_id")).distinct().count()

res7: Long = 3673


In [15]:
//4
data.map(_("start_station_id")).union(data.map(_("end_station_id"))).distinct().count()

res8: Long = 272


In [16]:
//5
val bike_rental_max = data.map(x => (x("bike_id"), x("duration_sec").asInstanceOf[Int])
                              ).reduceByKey(_ + _).max()(Ordering.by[(Any, Int), Int](_._2))

val bike_rental_min = data.map(x => (x("bike_id"), x("duration_sec").asInstanceOf[Int])
                              ).reduceByKey(_ + _).min()(Ordering.by[(Any, Int), Int](_._2))

bike_rental_max: (Any, Int) = (775,555010)
bike_rental_min: (Any, Int) = (722,70)


In [17]:
//6
val avg_rental = data.map(_("duration_sec").asInstanceOf[Int]).mean()

avg_rental: Double = 1099.0095208774264


In [18]:
//7
val most_popular_stations = data.map{
  case x if x("start_station_id").asInstanceOf[Int] > x("end_station_id").asInstanceOf[Int] => (x("start_station_id") + "-" + x("end_station_id"), 1)
  case x => (x("end_station_id") + "-" + x("start_station_id"), 1)
}.reduceByKey(_ + _).takeOrdered(3)(Ordering[Int].reverse.on(_._2))

most_popular_stations: Array[(String, Int)] = Array((15-6,5078), (16-6,3154), (81-15,3087))


In [19]:
//8
val rush_hour = data.map(x => (x("start_time").asInstanceOf[java.time.LocalDateTime].getHour, 1)
                        ).reduceByKey(_ + _).collect()

rush_hour: Array[(Int, Int)] = Array((4,640), (16,41339), (0,2606), (8,57227), (12,27448), (20,16642), (13,27374), (21,11993), (1,1377), (17,60111), (9,45457), (5,2145), (22,7967), (14,24900), (6,8895), (18,46088), (10,25217), (2,860), (19,26598), (15,28448), (11,24303), (23,4867), (3,398), (7,26800))


In [20]:
//9
  val intiaValue = (0, 0)
  val counter = (acc: (Int, Int), x: Int) => (acc._1 + x, acc._2 +1)
  val combCounter = (acc1: (Int, Int), acc2: (Int, Int)) => (acc1._1 + acc2._1, acc1._2 + acc2._2)

  val rush_days = data.map(x => (x("start_time").asInstanceOf[java.time.LocalDateTime].toLocalDate, 1)
  ).reduceByKey(_ + _ ).map(x => (x._1.getDayOfWeek, x._2)
  ).aggregateByKey(intiaValue)(counter, combCounter
  ).mapValues(x => x._1 / x._2).takeOrdered(7)

rush_days.foreach(println)

(MONDAY,3131)
(TUESDAY,3379)
(WEDNESDAY,3250)
(THURSDAY,3157)
(FRIDAY,3006)
(SATURDAY,1884)
(SUNDAY,1681)


intiaValue: (Int, Int) = (0,0)
counter: ((Int, Int), Int) => (Int, Int) = $Lambda$3166/0x000000084112d040@7ac344e0
combCounter: ((Int, Int), (Int, Int)) => (Int, Int) = $Lambda$3167/0x000000084112d840@3e61903b
rush_days: Array[(java.time.DayOfWeek, Int)] = Array((MONDAY,3131), (TUESDAY,3379), (WEDNESDAY,3250), (THURSDAY,3157), (FRIDAY,3006), (SATURDAY,1884), (SUNDAY,1681))


> **ZADANIE 2**: Utwórz RDD `dataDaily` zawierające dane zagregowane do poziomu dnia. Każdy dzień w roku (element w RDD) ma zawierać następujące informacje: 
- 'date' : data 
- 'avg_duration_sec' : średni czas wypożyczeń danego dnia
- 'n_trips' : liczba wypożyczeń danego dnia
- 'n_bikes' : liczba unikatowych rowerów użytych danego dnia
- 'n_routes' : liczba unikatowych kombinacji stacji (x -> y == y -> x) danego dnia
- 'n_subscriber' : liczba wypożyczeń dokonanych przez subskrybentów danego dnia

In [21]:
//avg_duration and n_trips
val avg_time_trip = data.map(x => (x("start_time").asInstanceOf[LocalDateTime].toLocalDate, x("duration_sec").asInstanceOf[Int])
  ).aggregateByKey(intiaValue)(counter, combCounter
  ).mapValues(x => (x._1 /x._2, x._2))

avg_time_trip.take(5)

avg_time_trip: org.apache.spark.rdd.RDD[(java.time.LocalDate, (Int, Int))] = MapPartitionsRDD[46] at mapValues at <console>:36
res10: Array[(java.time.LocalDate, (Int, Int))] = Array((2017-08-12,(1671,1945)), (2017-07-08,(2088,659)), (2017-11-12,(1528,2075)), (2017-11-20,(826,4049)), (2017-12-24,(1260,819)))


In [22]:
 //number of unique bikes
val uniq_bikes = data.map(x => (x("start_time").asInstanceOf[LocalDateTime].toLocalDate, x("bike_id"))).distinct(
).mapValues(x => 1).reduceByKey(_ + _)

uniq_bikes.take(5)

uniq_bikes: org.apache.spark.rdd.RDD[(java.time.LocalDate, Int)] = ShuffledRDD[52] at reduceByKey at <console>:31
res11: Array[(java.time.LocalDate, Int)] = Array((2017-07-08,305), (2017-08-12,937), (2017-07-24,752), (2017-10-16,1397), (2017-10-20,1310))


In [23]:
  //number of unique station
  val routes = data.map{
    case x if x("start_station_id").asInstanceOf[Int] > x("end_station_id").asInstanceOf[Int] => (x("start_time").asInstanceOf[LocalDateTime].toLocalDate, x("start_station_id") + "-" + x("end_station_id"))
    case x => (x("start_time").asInstanceOf[LocalDateTime].toLocalDate, x("end_station_id") + "-" + x("start_station_id"))
  }.distinct().mapValues(x => 1).reduceByKey(_ + _)

routes.take(5)

routes: org.apache.spark.rdd.RDD[(java.time.LocalDate, Int)] = ShuffledRDD[58] at reduceByKey at <console>:33
res12: Array[(java.time.LocalDate, Int)] = Array((2017-07-08,276), (2017-08-12,933), (2017-07-24,826), (2017-10-16,1974), (2017-10-20,1883))


In [24]:
  //number of subscribers
  val subscribers = data.filter(_("user_type") == "Subscriber").map(x =>(x("start_time").asInstanceOf[LocalDateTime].toLocalDate, 1)
  ).reduceByKey(_ + _)

subscribers.take(5)

subscribers: org.apache.spark.rdd.RDD[(java.time.LocalDate, Int)] = ShuffledRDD[61] at reduceByKey at <console>:31
res13: Array[(java.time.LocalDate, Int)] = Array((2017-08-12,813), (2017-07-08,281), (2017-11-12,1335), (2017-11-20,3540), (2017-12-24,573))


In [25]:
val dataDaily = avg_time_trip.join(uniq_bikes).mapValues(x => (x._1._1, x._1._2, x._2)
                            ).join(routes).mapValues(x => (x._1._1, x._1._2, x._1._3, x._2)
                            ).join(subscribers).mapValues(x => (x._1._1, x._1._2, x._1._3, x._1._4, x._2))

dataDaily.take(1).mkString(" ")

dataDaily: org.apache.spark.rdd.RDD[(java.time.LocalDate, (Int, Int, Int, Int, Int))] = MapPartitionsRDD[73] at mapValues at <console>:37
res14: String = (2017-08-12,(1671,1945,937,933,813))


In [26]:
  val daily = dataDaily.map(x => Map(
    "date" -> x._1,
    "avg_duration_sec" -> x._2._1,
    "n_trips" -> x._2._2,
    "n_bikes" -> x._2._3,
    "n_routes" -> x._2._4,
    "n_subscribers" -> x._2._5
  ))

daily.take(2).mkString("\n")

daily: org.apache.spark.rdd.RDD[scala.collection.immutable.Map[String,Any]] = MapPartitionsRDD[74] at map at <console>:29
res15: String =
Map(n_routes -> 933, date -> 2017-08-12, n_trips -> 1945, n_bikes -> 937, n_subscribers -> 813, avg_duration_sec -> 1671)
Map(n_routes -> 276, date -> 2017-07-08, n_trips -> 659, n_bikes -> 305, n_subscribers -> 281, avg_duration_sec -> 2088)


In [27]:
import java.time.LocalDate

import java.time.LocalDate


In [28]:
daily.filter(_("date") == java.time.LocalDate.of(2017, 12, 21)).collect().foreach(println)

Map(n_routes -> 1605, date -> 2017-12-21, n_trips -> 2995, n_bikes -> 1321, n_subscribers -> 2724, avg_duration_sec -> 743)


> **⋆ ZADANIE 3**: Utwórz RDD klucz-wartość `bikeDaily` zawierające po jednym elemencie dla każdego `"bike_id"`. Wartościami w RDD mają być listy całkowitego dziennego wykorzystania danego roweru w sekundach (elementy listy uszeregowane chronologicznie). 
- kryterium zaliczenia danego wypożyczenia do konkretnego dnia jest moment startu
- wszystkie rowery mają mieć listy jednakowej długości

In [29]:
val test = data.map(x => ((x("bike_id"), x("start_time").asInstanceOf[LocalDateTime].toLocalDate), 
                          x("duration_sec").asInstanceOf[Int])
            ).reduceByKey(_ + _)

test.take(1)

test: org.apache.spark.rdd.RDD[((Any, java.time.LocalDate), Int)] = ShuffledRDD[77] at reduceByKey at <console>:32
res17: Array[((Any, java.time.LocalDate), Int)] = Array(((1172,2017-09-17),504))


In [30]:
val first_day = data.map(x => x("start_time").asInstanceOf[LocalDateTime].toLocalDate.toEpochDay).min()

first_day: Long = 17345


In [34]:
val min_date = LocalDate.ofEpochDay(first_day)

min_date: java.time.LocalDate = 2017-06-28


In [33]:
val last_day = data.map(x => x("start_time").asInstanceOf[LocalDateTime].toLocalDate.toEpochDay).max()
val time_delta = last_day - first_day

last_day: Long = 17531
time_delta: Long = 186


In [36]:
val date_list = (0 to time_delta.toInt).map(x => min_date.plusDays(x))

date_list: scala.collection.immutable.IndexedSeq[java.time.LocalDate] = Vector(2017-06-28, 2017-06-29, 2017-06-30, 2017-07-01, 2017-07-02, 2017-07-03, 2017-07-04, 2017-07-05, 2017-07-06, 2017-07-07, 2017-07-08, 2017-07-09, 2017-07-10, 2017-07-11, 2017-07-12, 2017-07-13, 2017-07-14, 2017-07-15, 2017-07-16, 2017-07-17, 2017-07-18, 2017-07-19, 2017-07-20, 2017-07-21, 2017-07-22, 2017-07-23, 2017-07-24, 2017-07-25, 2017-07-26, 2017-07-27, 2017-07-28, 2017-07-29, 2017-07-30, 2017-07-31, 2017-08-01, 2017-08-02, 2017-08-03, 2017-08-04, 2017-08-05, 2017-08-06, 2017-08-07, 2017-08-08, 2017-08-09, 2017-08-10, 2017-08-11, 2017-08-12, 2017-08-13, 2017-08-14, 2017-08-15, 2017-08-16, 2017-08-17, 2017-08-18, 2017-08-19, 2017-08-20, 2017-08-21, 2017-08-22, 2017-08-23, 2017-08-24, 2017-08-25, 2017-08-26...


In [38]:
val bike_list = data.map(_("bike_id")).distinct().collect()

bike_list: Array[Any] = Array(1084, 772, 3272, 752, 1724, 428, 1900, 1328, 464, 1040, 1336, 912, 140, 204, 956, 1128, 2892, 3436, 228, 1416, 192, 492, 160, 1148, 1168, 1596, 1780, 2716, 1304, 3192, 992, 2904, 3240, 1500, 460, 2664, 2608, 3336, 548, 1404, 1732, 2092, 528, 696, 2108, 1632, 3032, 1716, 1308, 1808, 2712, 2912, 1280, 2052, 2116, 1816, 196, 2192, 1104, 784, 2732, 848, 3296, 3568, 1144, 1176, 292, 1592, 1296, 2180, 2164, 344, 1344, 1320, 664, 252, 2604, 2464, 1020, 3500, 2040, 988, 2360, 688, 2600, 968, 2964, 592, 1468, 1940, 836, 1556, 2924, 1800, 1932, 3484, 1656, 2744, 740, 436, 208, 2808, 3700, 3516, 3600, 1968, 804, 2792, 2768, 3520, 3000, 1172, 3616, 3128, 2588, 1424, 1044, 1076, 536, 2648, 1100, 3364, 3696, 2488, 3488, 3180, 2844, 1300, 2976, 3452, 672, 2248, 1540, 1680...


In [46]:
val comb_list = for{ bike <- bike_list; date <- date_list} yield ((bike, date), 0)

comb_list: Array[((Any, java.time.LocalDate), Int)] = Array(((1084,2017-06-28),0), ((1084,2017-06-29),0), ((1084,2017-06-30),0), ((1084,2017-07-01),0), ((1084,2017-07-02),0), ((1084,2017-07-03),0), ((1084,2017-07-04),0), ((1084,2017-07-05),0), ((1084,2017-07-06),0), ((1084,2017-07-07),0), ((1084,2017-07-08),0), ((1084,2017-07-09),0), ((1084,2017-07-10),0), ((1084,2017-07-11),0), ((1084,2017-07-12),0), ((1084,2017-07-13),0), ((1084,2017-07-14),0), ((1084,2017-07-15),0), ((1084,2017-07-16),0), ((1084,2017-07-17),0), ((1084,2017-07-18),0), ((1084,2017-07-19),0), ((1084,2017-07-20),0), ((1084,2017-07-21),0), ((1084,2017-07-22),0), ((1084,2017-07-23),0), ((1084,2017-07-24),0), ((1084,2017-07-25),0), ((1084,2017-07-26),0), ((1084,2017-07-27),0), ((1084,2017-07-28),0), ((1084,2017-07-29),0), (...


In [47]:
val helper_rdd = sc.parallelize(comb_list)
helper_rdd.take(2)

helper_rdd: org.apache.spark.rdd.RDD[((Any, java.time.LocalDate), Int)] = ParallelCollectionRDD[89] at parallelize at <console>:31
res22: Array[((Any, java.time.LocalDate), Int)] = Array(((1084,2017-06-28),0), ((1084,2017-06-29),0))


In [50]:
val temp_rdd = test.union(helper_rdd).map(x => (x._1._1,(x._1._2, x._2)))
temp_rdd.take(2)

temp_rdd: org.apache.spark.rdd.RDD[(Any, (java.time.LocalDate, Int))] = MapPartitionsRDD[93] at map at <console>:34
res24: Array[(Any, (java.time.LocalDate, Int))] = Array((1172,(2017-09-17,504)), (1829,(2017-09-14,2978)))


In [56]:
temp_rdd.groupByKey().mapValues(x => x.toList.sortBy(_._1.toEpochDay)).collect()                            

res29: Array[(Any, List[(java.time.LocalDate, Int)])] = Array((3272,List((2017-06-28,0), (2017-06-29,0), (2017-06-30,0), (2017-07-01,0), (2017-07-02,0), (2017-07-03,0), (2017-07-04,0), (2017-07-05,0), (2017-07-06,0), (2017-07-07,0), (2017-07-08,0), (2017-07-09,0), (2017-07-10,0), (2017-07-11,0), (2017-07-12,0), (2017-07-13,0), (2017-07-14,0), (2017-07-15,0), (2017-07-16,0), (2017-07-17,0), (2017-07-18,0), (2017-07-19,0), (2017-07-20,0), (2017-07-21,0), (2017-07-22,0), (2017-07-23,0), (2017-07-24,0), (2017-07-25,0), (2017-07-26,0), (2017-07-27,0), (2017-07-28,0), (2017-07-29,0), (2017-07-30,0), (2017-07-31,0), (2017-08-01,0), (2017-08-02,0), (2017-08-03,0), (2017-08-04,0), (2017-08-05,0), (2017-08-06,0), (2017-08-07,0), (2017-08-08,0), (2017-08-09,0), (2017-08-10,0), (2017-08-11,0), (201...


### Przetwarzanie tekstu
[Project Gutenberg](http://www.gutenberg.org/)
> **ZADANIE 4**: Wstępnie przetwórz i zbadaj Moby Dicka
1. usuń puste linie, tytuł i nazwy rozdziałów, wynik zapisz jako `textOnlyMD`
2. policz ile słów znajduje się w tekście
3. policz ile unikalnych słów występuje w tekście
4. znajdź 10 najczęstszych słów
5. sprawdź jak często wystepowało slowo "whale"

In [5]:
val text = sc.textFile("MobyDick.txt")
text.take(10)

text: org.apache.spark.rdd.RDD[String] = MobyDick.txt MapPartitionsRDD[7] at textFile at <console>:27
res3: Array[String] = Array(MOBY DICK;, "", "", or, THE WHALE., "", "", CHAPTER 1. Loomings., "", Call me Ishmael. Some years ago—never mind how long precisely—having, little or no money in my purse, and nothing particular to interest me on)


In [6]:
text.take(10).foreach(println)

MOBY DICK;


or, THE WHALE.


CHAPTER 1. Loomings.

Call me Ishmael. Some years ago—never mind how long precisely—having
little or no money in my purse, and nothing particular to interest me on


In [13]:
val header = text.take(6).toSet

header: scala.collection.immutable.Set[String] = Set(MOBY DICK;, "", or, THE WHALE.)


In [15]:
//1
val textOnlyMD = text.filter(!header.contains(_))

textOnlyMD.take(10).foreach(println)

CHAPTER 1. Loomings.
Call me Ishmael. Some years ago—never mind how long precisely—having
little or no money in my purse, and nothing particular to interest me on
shore, I thought I would sail about a little and see the watery part of
the world. It is a way I have of driving off the spleen and regulating
the circulation. Whenever I find myself growing grim about the mouth;
whenever it is a damp, drizzly November in my soul; whenever I find
myself involuntarily pausing before coffin warehouses, and bringing up
the rear of every funeral I meet; and especially whenever my hypos get
such an upper hand of me, that it requires a strong moral principle to


textOnlyMD: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[11] at filter at <console>:31


In [25]:
//2
textOnlyMD.flatMap(_.split("\\W+")).count()

res15: Long = 219137


In [27]:
//2 other approach by regex
import scala.util.matching.Regex

val pattern = new Regex("[a-zA-Z]+")

import scala.util.matching.Regex
pattern: scala.util.matching.Regex = [a-zA-Z]+


In [29]:
textOnlyMD.flatMap(x => pattern.findAllIn(x.toLowerCase)).count()

res17: Long = 217676


In [31]:
//3
textOnlyMD.flatMap(x => pattern.findAllIn(x.toLowerCase)).distinct().count()

res19: Long = 16888


In [37]:
//4
textOnlyMD.flatMap(x => pattern.findAllIn(x.toLowerCase)).map(x => (x,1)).reduceByKey(_ + _
).takeOrdered(10)(Ordering[Int].reverse.on(_._2)).foreach(println)

(the,14348)
(of,6583)
(and,6393)
(a,4695)
(to,4619)
(in,4135)
(that,3060)
(it,2508)
(his,2495)
(i,2114)


In [38]:
//5 occurance of word whale
textOnlyMD.flatMap(x => pattern.findAllIn(x.toLowerCase)).filter(_ == "whale").count()

res26: Long = 1152
